In [2]:
import pandas as pd
import json
import os

data = pd.read_csv('data/results.csv')

In [3]:
data['date'] = pd.to_datetime(data['date'])
years = data['date'].dt.year.unique()
print(years)

[2020 2019 2018 2017 2016 2015]


In [4]:
all_result = []

for year in years:

    df_year = data[data['date'].dt.year == year]

    summary = []

    for index, row in df_year.iterrows():
        team1 = row['team_1']
        team2 = row['team_2']
        map_name = row['_map']
        t1_score = row['t_1']
        ct2_score = row['ct_2']
        ct1_score = row['ct_1']
        t2_score = row['t_2']

        summary.append([team1, map_name, 'T', t1_score > ct2_score])
        summary.append([team2, map_name, 'CT', ct2_score > t1_score])
        summary.append([team1, map_name, 'CT', ct1_score > t2_score])
        summary.append([team2, map_name, 'T', t2_score > ct1_score])

    results_df = pd.DataFrame(summary, columns=['team', 'map', 'side', 'win'])

    win_data = results_df.groupby(['team', 'map', 'side']).agg(
    total_matches=('win', 'size'),
    wins=('win', 'sum')
    ).reset_index()

    win_data['win_rate'] = win_data['wins'] / win_data['total_matches']

    final = win_data.pivot_table(index=['team', 'map'], columns='side', values=['total_matches', 'wins']).reset_index()
    final.columns = ['team', 'map', 'CT_total_matches', 'T_total_matches', 'CT_wins', 'T_wins']
    final['CT_win_rate'] = final['CT_wins'] / final['CT_total_matches']
    final['T_win_rate'] = final['T_wins'] / final['T_total_matches']
    final['year'] = year

    all_result.append(final)

final_results = pd.concat(all_result)
print(final_results)

            team       map  CT_total_matches  T_total_matches  CT_wins  \
0    100 Thieves     Dust2               4.0              4.0      2.0   
1    100 Thieves   Inferno               4.0              4.0      1.0   
2    100 Thieves    Mirage               2.0              2.0      1.0   
3    100 Thieves      Nuke               3.0              3.0      2.0   
4    100 Thieves     Train               1.0              1.0      0.0   
..           ...       ...               ...              ...      ...   
175  mousesports     Dust2               2.0              2.0      1.0   
176  mousesports   Inferno               2.0              2.0      1.0   
177  mousesports    Mirage               4.0              4.0      4.0   
178  mousesports  Overpass               2.0              2.0      2.0   
179  mousesports     Train               1.0              1.0      0.0   

     T_wins  CT_win_rate  T_win_rate  year  
0       2.0     0.500000    0.500000  2020  
1       2.0     0.250

In [5]:
minimum = final_results['T_total_matches'].min()
maximum = final_results['T_total_matches'].max()
print(minimum,maximum)

1.0 86.0


In [6]:
map_data = final_results.to_json(orient='records')

with open('data/map_statistics.json', 'w') as f:
    f.write(map_data)